In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('sentiment.csv')

In [ ]:
import openai
api_key = ""
openai.api_key = api_key

In [4]:
df = data[:1000]

In [5]:
import pandas as pd
import openai

# Hàm để đánh giá thông tin bằng GPT
def evaluate_vn30_impact(text):
    prompt = f"""Đánh giá đoạn dữ liệu này: '{text}' có ảnh hưởng như thế nào đến tâm lý nhà đầu tư trên hợp đồng tương lai VN30. Nếu cảm thấy thông tin có thể giúp hợp đồng tăng giá thì trả lời số '1', nếu có thể làm giảm giá thì trả lời '-1', còn nếu không có tác động rõ ràng, trả lời '0'."""

    # Sử dụng ChatCompletion để gửi prompt tới GPT
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-2024-08-06",  # Hoặc "gpt-4" nếu bạn muốn sử dụng mô hình GPT-4
            messages=[
                {"role": "system", "content": "You are a financial analyst."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10,  # Số lượng token tối đa cho phản hồi
            temperature=0  # Đặt nhiệt độ thấp để có kết quả nhất quán hơn
        )
        
        # Lấy phản hồi từ GPT
        answer = response['choices'][0]['message']['content'].strip()
        
        # Chuyển đổi phản hồi thành số (-1, 0, 1)
        return int(answer)
    
    except (ValueError, IndexError):
        return None

# Thêm cột 'price movement' vào DataFrame
df['price movement'] = df['Text'].apply(evaluate_vn30_impact)

# Lưu kết quả vào file CSV mới
df.to_csv('articles_with_price_movement.csv', index=False, encoding='utf-8')

# Hiển thị kết quả
print(df)


                                                  Text  price movement
0    Điều ít biết về sàn TMĐT Temu đang gây bão vì ...             0.0
1    Margin cao kỷ lục nhưng thanh khoản thị trường...            -1.0
2    Tự doanh CTCK trở lại mua ròng trong phiên 24/...             1.0
3    Tỷ phú Nguyễn Đăng Quang chỉ nắm 18 cổ phiếu M...             0.0
4    Chuỗi siêu thị của tỉ phú Nguyễn Đăng Quang bắ...             1.0
..                                                 ...             ...
995  HoSE tiếp tục cắt margin cổ phiếu do thua lỗ, ...            -1.0
996  Biến động nhân sự cấp cao tại Đầu tư Hải Phát ...             0.0
997  “The Investors”: Chuỗi Talkshow truyền cảm hứn...             NaN
998  Hé lộ thu nhập "khủng" của chủ tịch, tổng giám...             0.0
999  ‘Sếp lớn’ Thế giới Di động bán mạnh cổ phiếu M...            -1.0

[1000 rows x 2 columns]


C:\Users\Minh Truc\AppData\Local\Temp\ipykernel_17892\378446692.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price movement'] = df['Text'].apply(evaluate_vn30_impact)


Nếu lấy xu hướng giá làm output cho news, cần chuẩn bị:
- Sử dụng chatgpt4 để đánh giá với thông tin news, xu hướng giá vn30 sẽ như thế nào. (1)
- Khung giờ giao dịch là 8h45 & 9h-11h30, 13h-14h30 & 14h45, do đó phải tách dữ liệu vào các khung thời gian này.
- Ở các dữ liệu ngoài khung thời gian giao dịch, ta không biết xu hướng giá thật sự phản ứng dựa theo thông tin nào trong vô số thông tin, ta sẽ lấy (1) làm output. (1.5)
- Ở các dữ liệu trong các khung thời gian giao dịch và gần đó cụ thể là khoảng 7h30- 14h30, với giả định đa số nhà đầu tư sẽ phản ứng sớm với thông tin, ta xem xét xu hướng giá (price percentage change) của 1', 5', 10', 60' sau, nếu lớn hơn 75% sẽ cho là 1, nhỏ hơn 25% sẽ cho là -1, còn lại cho 0. (2)
- Xem xét các correlation giữa (2) và (1), 



# For concat

In [ ]:
df2= pd.read_csv('articles_with_price_movement.csv')

In [ ]:
df_new= pd.concat([df2,df], axis=0, ignore_index=True)

In [ ]:
df_new.to_csv('1000.csv',index=False)